# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-CU hangs when receiving F1AP Setup Request with mutated DU-Served-Cell-List.
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-CU hangs when receiving F1AP Setup Request with mutated gNB-DU Served Cells List; attacker also mutates UE Context Modification (RNTI) and sends DL RRC Message Transfer post UE release.",
    "interface_guess": ["F1AP", "RRC"],
    "procedure_guess": ["F1 Setup", "UE Context Modification", "DL RRC Message Transfer"],
    "components_involved": ["gNB-CU", "gNB-DU", "UE"],
    "key_ids": {
      "transaction_id": "",
      "cu_ue_f1ap_id": "",
      "du_ue_f1ap_id": "",
      "du_id": "",
      "rnti": "modified/invalid",
      "pci": "",
      "served_cell_list": ["mutated"]
    },
    "signals_or_timers": ["F1 Setup Request", "UE Context Modification Required", "DL RRC Message Transfer"]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {"Keywords": ["DL RRC", "UE-associated", "IDs"], "title": ["8.4.2 DL RRC Message Transfer", "8.4.2.2 Successful operation"]},
    {"Keywords": ["UE Context Modification Required", "IDs", "Cause"], "title": ["9.2.2.10 UE CONTEXT MODIFICATION REQUIRED"]},
    {"Keywords": ["UE Context Modification Failure"], "title": ["9.2.2.9 UE CONTEXT MODIFICATION FAILURE"]},
    {"Keywords": ["UE Context Modification Confirm"], "title": ["9.2.2.11 UE CONTEXT MODIFICATION CONFIRM"]},
    {"Keywords": ["F1 Setup", "Served Cells List", "gNB-DU"], "title": ["8.2.3.2 Successful Operation (F1 Setup)"]}
  ],
  "expected_behaviour": "CU validates F1 Setup Request IEs (incl. gNB-DU Served Cells List) per TS 38.473/38.401 and must not hang on malformed input; UE Context Modification Required includes gNB-CU and gNB-DU UE F1AP IDs and Cause; DL RRC Message Transfer is CU->DU and requires UE F1AP IDs and RRC-Container."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {"path": "F1AP/f1ap_cu_interface_management.c", "function_name": "CU_handle_F1_SETUP_REQUEST", "reason": "CU-side decoding and forwarding of F1 Setup Request."},
    {"path": "F1AP/lib/f1ap_interface_management.c", "function_name": "encode_f1ap_setup_request", "reason": "Constructs gNB-DU Served Cells List; related decode handles parsing and may hit malformed list."},
    {"path": "F1AP/f1ap_cu_ue_context_management.c", "function_name": "CU_handle_UE_CONTEXT_MODIFICATION_REQUIRED", "reason": "Parses mandatory gNB-CU/DU UE F1AP IDs and Cause."},
    {"path": "F1AP/f1ap_cu_rrc_message_transfer.c", "function_name": "CU_send_DL_RRC_MESSAGE_TRANSFER", "reason": "Builds and sends DL RRC Message Transfer from CU to DU."},
    {"path": "F1AP/f1ap_handlers.c", "function_name": "F1Setup mapping", "reason": "Elementary procedure mapping to CU/DU handlers for F1Setup."}
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {"kind": "spec", "source": "F1AP.docx", "location": "8.4.2.2", "text": "CU initiates DL RRC MESSAGE TRANSFER; if UE-associated logical F1-connection exists, include gNB-DU UE F1AP ID. If none exists, it is established at reception."},
    {"kind": "spec", "source": "F1AP.docx", "location": "9.2.2.10", "text": "UE CONTEXT MODIFICATION REQUIRED includes mandatory gNB-CU UE F1AP ID, gNB-DU UE F1AP ID and Cause; optional DU to CU RRC Information."},
    {"kind": "spec", "source": "F1AP.docx", "location": "8.2.3.2", "text": "If F1 SETUP REQUEST contains the gNB-DU Served Cells List IE, the gNB-CU shall take it into account as specified in TS 38.401."},
    {"kind": "code", "source": "F1AP/f1ap_cu_interface_management.c", "location": "L86-L111", "text": "CU_handle_F1_SETUP_REQUEST: checks stream==0, decodes F1AP Setup Request via decode_f1ap_setup_request, then ITTI to RRC."},
    {"kind": "code", "source": "F1AP/lib/f1ap_interface_management.c", "location": "L756-L772", "text": "In encode_f1ap_setup_request, gNB_DU_Served_Cells_List is created and each GNB_DU_Served_Cells_Item is filled from msg->cell[]."},
    {"kind": "code", "source": "F1AP/f1ap_cu_rrc_message_transfer.c", "location": "L69-L90", "text": "CU_send_DL_RRC_MESSAGE_TRANSFER: encodes DL RRC, PER-encodes PDU, sends to DU over SCTP."},
    {"kind": "code", "source": "F1AP/f1ap_cu_ue_context_management.c", "location": "L186-L193", "text": "CU_handle_UE_CONTEXT_MODIFICATION_REQUIRED: extracts GNB_CU_UE_F1AP_ID and GNB_DU_UE_F1AP_ID from IEs."}
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-DU", "to": "gNB-CU", "message": "F1 SETUP REQUEST (mutated Served Cells List)", "precond": "SCTP association up", "postcond": "CU decodes and validates"},
    {"from": "gNB-CU", "to": "gNB-CU", "message": "Decode F1 Setup; expected: validate Served Cells per 38.401; observed: hang", "postcond": "No response"},
    {"from": "Attacker", "to": "gNB-CU", "message": "UE Context Modification (mutated RNTI)", "precond": "Forged/altered message"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UE Context Release Complete", "precond": "UE released"},
    {"from": "Attacker", "to": "gNB-DU", "message": "DL RRC MESSAGE TRANSFER (forged)", "precond": "Post-release injection"}
  ],
  "state_machine": {
    "states": ["Idle", "AwaitingSetup", "Connected", "UEContextActive", "UEContextReleased", "Error/Hang"],
    "transitions": [
      {"from": "Idle", "to": "AwaitingSetup", "on": "SCTP up"},
      {"from": "AwaitingSetup", "to": "Connected", "on": "F1 SETUP RESPONSE", "guard": "Valid Setup Request"},
      {"from": "AwaitingSetup", "to": "Error/Hang", "on": "Malformed F1 SETUP REQUEST", "guard": "Invalid Served Cells List"},
      {"from": "Connected", "to": "UEContextActive", "on": "UE CONTEXT SETUP"},
      {"from": "UEContextActive", "to": "UEContextReleased", "on": "UE CONTEXT RELEASE"},
      {"from": "UEContextReleased", "to": "Error/Hang", "on": "Forged DL RRC MESSAGE TRANSFER", "guard": "Direction/ID mismatch"}
    ]
  },
  "assumptions": [
    "Hang occurs in F1 Setup decode path when Served Cells List is malformed.",
    "Attacker can inject or alter F1AP messages on control path.",
    "F1AP does not carry RNTI directly; the 'RNTI' mention maps to UE F1AP IDs or RRC payload content."
  ],
  "questions": [
    "Exact malformed field within GNB_DU_Served_Cells_List that triggers the hang?",
    "Do logs show decode_f1ap_setup_request stuck or ITTI backpressure?",
    "Was DL RRC MESSAGE TRANSFER observed on DU side, indicating spoofing?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {"name": "DL RRC is CU->DU and requires UE F1AP IDs", "result": "FAIL", "evidence": ["Spec 8.4.2.2: CU initiates; includes gNB-DU UE F1AP ID", "Code F1AP/f1ap_cu_rrc_message_transfer.c L69-L90"]},
    {"name": "UE Context Modification Required contains both UE F1AP IDs", "result": "PASS", "evidence": ["Spec 9.2.2.10: mandatory gNB-CU UE F1AP ID and gNB-DU UE F1AP ID", "Code F1AP/f1ap_cu_ue_context_management.c L186-L193"]},
    {"name": "CU must not hang on malformed F1 Setup Served Cells List", "result": "FAIL", "evidence": ["Spec 8.2.3.2: CU shall take Served Cells List into account (robust handling implied)", "Code F1AP/f1ap_cu_interface_management.c L86-L111 (decoding path)", "Bug observation: CU hang when list is mutated"]}
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "The CU hangs when decoding a malformed F1 SETUP REQUEST (mutated gNB-DU Served Cells List), indicating insufficient input validation or decode-loop robustness. Additionally, DL RRC MESSAGE TRANSFER spoofing from DU violates directionality per spec."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "8.4.2.2 DL RRC Message Transfer (Successful operation)",
      "9.2.2.10 UE CONTEXT MODIFICATION REQUIRED",
      "8.2.3.2 Successful Operation (F1 Setup: Served Cells List handling)"
    ],
    "code_citations": [
      "F1AP/f1ap_cu_interface_management.c:L86-L111",
      "F1AP/lib/f1ap_interface_management.c:L756-L772",
      "F1AP/f1ap_cu_rrc_message_transfer.c:L69-L90",
      "F1AP/f1ap_cu_ue_context_management.c:L186-L193"
    ],
    "expected_vs_observed": "Expected: CU decodes F1 SETUP REQUEST robustly and either accepts or rejects without hanging; UE Context Modification Required binds both UE F1AP IDs; DL RRC is CU->DU with valid IDs. Observed: CU hang on mutated Served Cells List; attacker scenario includes DU-originated DL RRC (direction violation)."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "gNB-CU hangs when decoding malformed F1 SETUP REQUEST (mutated gNB-DU Served Cells List), indicating missing validation/timeout. Directionality/ID-binding also require enforcement for UE-associated procedures.",
    "citations": {
      "spec": [
        "8.4.2.2 DL RRC Message Transfer (Successful operation)",
        "9.2.2.10 UE CONTEXT MODIFICATION REQUIRED",
        "8.2.3.2 Successful Operation (F1 Setup: Served Cells List handling)"
      ],
      "code": [
        "F1AP/f1ap_cu_interface_management.c:L86-L111",
        "F1AP/lib/f1ap_interface_management.c:L756-L772",
        "F1AP/f1ap_cu_rrc_message_transfer.c:L69-L90",
        "F1AP/f1ap_cu_ue_context_management.c:L186-L193"
      ]
    },
    "suggested_repro": [
      "Start gNB-CU and gNB-DU; ensure SCTP association is established.",
      "Send F1 SETUP REQUEST with malformed gNB-DU Served Cells List (e.g., missing mandatory IE, invalid count/length, duplicate items).",
      "Observe CU decode path responsiveness and logs; verify no hang and proper reject path.",
      "Attempt UE CONTEXT MODIFICATION REQUIRED with mismatched CU/DU UE F1AP IDs and verify rejection.",
      "Attempt DU-originated DL RRC MESSAGE TRANSFER and verify CU-side checks enforce direction/IDs."
    ],
    "risks": [
      "Control-plane DoS from CU hang on malformed setup.",
      "Potential acceptance of spoofed UE-associated messages if ID bindings aren’t enforced."
    ],
    "next_steps": [
      "Add strict validation and bounds checks in F1 Setup decode; reject invalid Served Cells List with failure cause.",
      "Introduce decode watchdog/timeout to prevent hangs; ensure graceful error paths.",
      "Enforce directionality and CU/DU UE F1AP ID binding for UE-associated procedures (e.g., DL RRC).",
      "Add unit/fuzz tests for Served Cells List parser and UE-associated message validators.",
      "Instrument detailed error logging around F1 Setup and UE context procedures for triage."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.